# Prediction of Unsteady Flow Field with Moving Boundary

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Overview

As an important tool to simulate and analyze fluid motion through numerical methods, CFD greatly facilitates the scientific research of fluid mechanics related issues, and plays an important role in providing accurate data and insights in the fields of design, optimization and research. One of the representative and research value problems in fluid mechanics is to simulate the unsteady flow field system with moving boundary to analyze the force of the moving structure in the flow field, which can optimize the design of the moving structure in engineering, and provide scheme strategies for the shape optimization of aerospace vehicles and navigation vehicles. High precision computational fluid dynamics (CFD) can accurately simulate the evolution of flow field and the stress of structure, but the high-precision dynamic boundary problem requires a large number of grids, which leads to huge hardware consumption and computational time cost. In addition, the construction of dynamic grids is also particularly time-consuming.

When CFD is applied to complex problems, the amount of calculation is huge and the calculation accuracy needs to be improved. An effective solution is given in the field of intelligent fluid mechanics. Deep learning can learn the evolution relationship between flow conditions and flow field through deep neural network, and quickly realize high-precision prediction and reconstruction of flow field. In order to efficiently solve the problem of reconstructing the flow field at the moving boundary, a hybrid depth neural network (hdnn) is proposed to reconstruct the unsteady flow field at the moving boundary, and fast predict the flow field based on it.

## Problem description

The relevant dimensions of the flow field are shown in the figure, where $Y = Asin(2πft)$ represents the motion expression of the cylinder in a simple harmonic motion in the vertical direction, a is the amplitude, f is the frequency; D stands for cylinder diameter; The rectangular boundary represents the computational domain. When the uniform incoming flow flows through a moving cylinder, under the influence of the interaction between the fluid and the solid, a series of complex flow phenomena will be formed behind the cylinder, such as boundary layer separation, alternating Karman vortex street and so on, and evolve into a non-uniform flow field whose physical quantities change periodically with time.

## Technology path

The specific process of mindflow to solve this problem is as follows:

1.Create data sets based on CFD numerical simulation results.

2.The model is built using mindspire deep learning framework.

3.Define the optimizer and loss function.

4.Use mindspire's instant compilation to accelerate model training.

5.Use the trained model for reasoning and visualization.

![p1.png](./images/p1.png)

## Model Architecture

The basic framework of HDNN consists of convolutional neural network (CNN), convolutional long short-term memory network (ConvLSTM) and deconvolution neural network (DeCNN). CNN reduces the dimensionality of the time series flow field and achieves feature extraction; ConvLSTM learns low dimensional spatiotemporal features and makes predictions; Finally, DeCNN achieves reconstruction of predicted flow fields

+ Input layer: Input historical flow field
+ Convolutional layer: Using multi-layer CNN to reduce the dimensionality of the input flow field and extract high-dimensional spatiotemporal flow characteristics
+ Memory layer: learning the evolution of spatiotemporal characteristics of low dimensional spatial flow fields through ConvLSTM and predicting the next moment
+ Deconvolution output layer: Restores the low-dimensional features of the predicted flow field to high-dimensional space, reconstructs the transient flow field at the next moment through multi-layer DeCNN, and outputs visual prediction results

![HDNN.jpg](./images/HDNN.jpg)

## Training dataset

The dataset is constructed from multidimensional matrix flow field snapshot matrix constructed from numerical simulation of unsteady two-dimensional cylindrical flow field data

+ A two-dimensional cylinder undergoes one-dimensional harmonic vibration in a uniform flow field, with vibration frequencies f (Hz) of 1.25, 1.43, 1.67, and 2.00, and amplitude ratios A/D of 0.5, 0.6, 0.7, and 0.8, respectively. Pairwise combination for a total of 16 sets of motion states
+ The dataset is a series of unsteady flow field data in a certain state (f, A/D)
+ Each flow field snapshot contains three channels, representing the pressure distribution information, horizontal velocity information, and vertical velocity information of the flow field. The size of the multi-dimensional matrix flow field snapshot matrix is: T × C × H × W (C is the number of channels, H, W are the height and width of the snapshot, respectively)
+ Dataset: [Download location](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/move_boundary_hdnn)

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, data_sink, jit
from mindflow.utils import load_yaml_config

from src import my_train_dataset, AEnet, save_loss_curve

## Training environment

+ The training adopts the static graphical model of Mindspot framework (GRAPH)
+ Train on CPU, GPU, or Ascend (single card)
+ The cylindrical vibration frequencies f (Hz) in the training dataset are 1.25, 1.43, and 1.67, respectively, and the amplitude ratios A/D are 0.5, 0.6, and 0.7, respectively. Pairwise combination for a total of 9 sets of motion states

In [2]:
set_seed(0)
np.random.seed(0)

## Training hyperparameter

Obtain hyperparameters for models, data, and optimizers from config

In [3]:
parser = argparse.ArgumentParser(description="cylinder around flow ROM")

parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./summary")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'GPU','Ascend'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--data_list", type=list, default=['0.00', '0.25', '0.35', '0.45'],
                    help="The type for training, [0.00, 0.25, 0.35, 0.45] for multi_state training /n"
                         "[0.25],....,[0.45] for single_state training")
parser.add_argument('--batch_size', type=int, default=16, help="mini batch_size")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")

args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs, save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target, device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

config = load_yaml_config(args.config_file_path)
data_params = config["data"]
model_params = config["model"]
optimizer_params = config["optimizer"]

## Training process file save path

Save the trained model file in a folder every certain number of training sessions

In [4]:
ckpt_dir = optimizer_params["ckpt_dir"]
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## Constructing neural network and optimizer

The convolutional layer of the neural network has a total of 12 layers, ConvLSTM has 1 layer, and deconvolution has a total of 12 layers

The Loss function uses the Mean squared error Loss function, and the optimizer uses the Adam (Adaptive Moment Estimation) optimization algorithm

In [5]:
model = AEnet(in_channels=model_params["in_channels"],
              num_layers=model_params["num_layers"],
              kernel_size=model_params["kernel_size"],
              num_convlstm_layers=model_params["num_convlstm_layers"])

loss_func = nn.MSELoss()
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=optimizer_params["lr"])
if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, 'O1')
else:
    loss_scaler = None

## Training framework

Define the forward propagation function forward_ Fn, compare the predicted value with the true value to obtain the loss value and return it

In [6]:
def forward_fn(inputs, velocity, label):
    pred = model(inputs, velocity)
    loss = loss_func(pred, label)

    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

## Dataset loading

To my_train_dataset parameter transfer to obtain training and validation datasets

In [7]:
print(f"==================Load data sample ===================")
dataset_train, dataset_eval = my_train_dataset(data_params["data_dir"],
                                               data_params["time_steps"],
                                               args.data_list)
print(f"======================End Load========================\n")

## Data sink and model training

Define train_ Step and Eval_ Step and use data_ Sink acceleration training, output the loss value and usage time during the training process, and save the model file every certain training round

In [8]:
print(f"====================Start train=======================")
@jit
def train_step(inputs, velocity, label):
    loss, grads = grad_fn(inputs, velocity, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, optimizer(grads))
    return loss

@jit
def eval_step(inputs, velocity, label):
    loss = forward_fn(inputs, velocity, label)
    loss = ops.sqrt(loss)
    return loss

train_sink_process = data_sink(train_step, dataset_train, sink_size=1)
eval_sink_process = data_sink(eval_step, dataset_eval, sink_size=1)
train_data_size, eval_data_size = dataset_train.get_dataset_size(), dataset_eval.get_dataset_size()

avg_train_losses = []
avg_valid_losses = []

for epoch in range(1, optimizer_params["epochs"] + 1):
    train_losses = 0
    valid_losses = 0

    local_time_beg = time.time()
    model.set_train(True)

    for _ in range(train_data_size):
        step_train_loss = ops.squeeze(train_sink_process(), axis=())
        step_train_loss = step_train_loss.asnumpy().item()
        train_losses += step_train_loss

    train_loss = train_losses / train_data_size
    avg_train_losses.append(train_loss)

    print(f"epoch: {epoch}, epoch average train loss: {train_loss :.6f}, "
          f"epoch time: {(time.time() - local_time_beg):.2f}s")

    if epoch % optimizer_params["eval_interval"] == 0:
        print(f"=================Start Evaluation=====================")

        eval_time_beg = time.time()
        model.set_train(False)
        for _ in range(eval_data_size):
            step_eval_loss = ops.squeeze(eval_sink_process(), axis=())
            step_eval_loss = step_eval_loss.asnumpy().item()
            valid_losses += step_eval_loss

        valid_loss = valid_losses / eval_data_size
        avg_valid_losses.append(valid_loss)

        print(f"epoch: {epoch}, epoch average valid loss: {valid_loss :.6f}, "
              f"epoch time: {(time.time() - eval_time_beg):.2f}s")
        print(f"==================End Evaluation======================")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/net_{epoch}.ckpt")

save_loss_curve(avg_train_losses, 'Epoch', 'avg_train_losses', 'Avg_train_losses Curve', 'Avg_train_losses.png')
save_loss_curve(avg_valid_losses, 'Epoch', 'avg_valid_losses', 'Avg_valid_losses Curve', 'Avg_valid_losses.png')

print(f"=====================End train========================")

## Set training conditions for parameter transmission

When running the file, pass in the necessary parameters through the parameter parser to start training, and print the process and device id, as well as the total training time

In [9]:
if __name__ == "__main__":
    print("Process ID:", os.getpid())
    print(f"device id: {args.device_id}")
    start_time = time.time()
    train()
    print(f"End-to-End total time: {(time.time() - start_time):.2f}s")

## Visualization of predicted flow field results

+ The moving boundary flow field prediction is started by executing eval.py, which can be divided into two prediction methods: single step flow field prediction (infer_mode is "one") and continuous flow field prediction within a vibration period (infer_mode is "cycle"); Single step flow field prediction only predicts the flow field of one time step at the next moment, while continuous flow field prediction continuously predicts the flow field of a complete cycle
+ The following figure shows the results of a fully trained HDNN model for one-step prediction and one complete cycle prediction of unsteady moving boundaries with a vibration frequency of 1.43Hz and an amplitude of 0.8 (amplitude ratio generalization state) (displaying pressure field, horizontal velocity field, and vertical velocity field)

![pred_single_step_puv.jpg](./images/pred_single_step_puv.jpg)

![pred_cycle_puv.jpg](./images/pred_cycle_puv.jpg)